In [52]:
import os
import pandas as pd
import numpy as np
import urllib
import plotly.graph_objects as go
import calendar

In [53]:
path = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\TempComparison\resample'
os.chdir(path)
csv = path + r'\csv.csv'
f = open(csv,'w+')

In [54]:
urllib.request.urlretrieve(
    'http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=daily&sids=rfd01' + 
    '&sdate=1992-06-04&edate=2018-12-31&elems=tmax,mxtm,tmin,mntm',filename=csv)
df = pd.read_csv(csv)
df = df.reset_index()

In [55]:
df.columns = ['station','date','tmax','mxtm','tmin','mntm']

df['tmax'] = (32 + (df.tmax * (9/5)))
df['tmin'] = (32 + (df.tmin * (9/5)))

In [56]:
df['tmax_time'] = (df.date + ' ' + df.mxtm)
df['tmin_time'] = (df.date + ' ' + df.mntm)

In [57]:
df['tmax_time'] = pd.to_datetime(df.tmax_time)
df['tmin_time'] = pd.to_datetime(df.tmin_time)

In [58]:
df['date'] = df.tmax_time.dt.date
df['date'] = pd.to_datetime(df.date)

In [59]:
tmin = ['station','tmin','tmin_time','date']
tmin_df = df[tmin]

tmax = ['station','tmax','tmax_time','date']
tmax_df = df[tmax]

In [60]:
tmax_resample = tmax_df.resample(rule='24H',label='right',on='tmax_time',base=17).max()
tmin_resample = tmin_df.resample(rule='24H',label='right',on='tmin_time',base=17).min()

In [61]:
stuff = ['tmin_time']
tmin_resample = tmin_resample.drop(stuff,axis=1)
tmin_resample = tmin_resample.reset_index()
tmin_resample = tmin_resample.drop(stuff,axis=1)

In [62]:
stuff = ['tmax_time']
tmax_resample = tmax_resample.drop(stuff,axis=1)
tmax_resample = tmax_resample.reset_index()
tmax_resample = tmax_resample.drop(stuff,axis=1)

In [63]:
tmin_resample.dtypes

station            object
tmin              float64
date       datetime64[ns]
dtype: object

In [64]:
urllib.request.urlretrieve(
    'http://data.rcc-acis.org/StnData?sid=057167&sdate=1992-06-05&edate=2018-12-31' + 
    '&elems=mint&output=csv',filename=csv)
coop_mint_df = pd.read_csv(csv)
coop_mint_df = coop_mint_df.reset_index()
coop_mint_df.columns = ['date','mint']
coop_mint_df = coop_mint_df.replace('M',np.nan,regex=True)

In [65]:
urllib.request.urlretrieve(
    'http://data.rcc-acis.org/StnData?sid=057167&sdate=1992-06-05&edate=2018-12-31' + 
    '&elems=maxt&output=csv',filename=csv)
coop_maxt_df = pd.read_csv(csv)
coop_maxt_df = coop_maxt_df.reset_index()
coop_maxt_df.columns = ['date','maxt']
coop_maxt_df = coop_maxt_df.replace('M',np.nan,regex=True)

In [66]:
tmin_comp_df = tmin_resample.merge(coop_mint_df,left_index=True,right_index=True)
tmax_comp_df = tmax_resample.merge(coop_maxt_df,left_index=True,right_index=True)

In [67]:
tmax_comp_df['maxt'] = tmax_comp_df.maxt.astype(float)
tmin_comp_df['mint'] = tmin_comp_df.mint.astype(float)

In [68]:
tmax_comp_df['diff'] = (tmax_comp_df.tmax - tmax_comp_df.maxt)
tmin_comp_df['diff'] = (tmin_comp_df.tmin - tmin_comp_df.mint)

In [69]:
stuff = ['date_y']
tmax_comp_df = tmax_comp_df.drop(stuff,axis=1)
tmin_comp_df = tmin_comp_df.drop(stuff,axis=1)

In [72]:
tmin_comp_df.columns = ['station','tmin','date','mint','diff']
tmax_comp_df.columns = ['station','tmax','date','maxt','diff']

In [78]:
tmax_comp_df = tmax_comp_df.dropna()
tmin_comp_df = tmin_comp_df.dropna()
tmin_comp_df['month'] = tmin_comp_df.date.dt.month
tmax_comp_df['month'] = tmax_comp_df.date.dt.month
tmin_comp_df['month'] = tmin_comp_df.month.astype(int)
tmax_comp_df['month'] = tmax_comp_df.month.astype(int)

C:\Users\student\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\student\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\student\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org

In [79]:
tmin_comp_df

,station,tmin,date,mint,diff,month
0,rfd01,52.8980,1992-06-04,53.0,-0.1020,6
1,rfd01,53.8880,1992-06-05,46.0,7.8880,6
2,rfd01,47.0840,1992-06-06,51.0,-3.9160,6
3,rfd01,52.7900,1992-06-07,53.0,-0.2100,6
4,rfd01,54.2300,1992-06-08,53.0,1.2300,6
5,rfd01,51.4040,1992-06-09,45.0,6.4040,6
6,rfd01,46.2200,1992-06-10,53.0,-6.7800,6
8,rfd01,51.4220,1992-06-11,56.0,-4.5780,6
9,rfd01,56.9480,1992-06-13,55.0,1.9480,6
10,rfd01,56.3720,1992-06-14,56.0,0.3720,6


In [80]:
tmin_comp_df['month'] = tmin_comp_df['month'].apply(lambda x: calendar.month_abbr[x])
tmax_comp_df['month'] = tmax_comp_df['month'].apply(lambda x: calendar.month_abbr[x])

C:\Users\student\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\student\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
tmax_fig = go.Figure(data=[go.Bar()],
                    layout=go.Layout(title=go.layout.Title(text='Average Monthly Max Temp Comparison'),
                                    xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='Month')),
                                    yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Temperature (°F)'))))
tmax_fig.add_trace(
    go.Bar(x=monthly.month,y=monthly.tmax,text=monthly.tmax_diff,name='RFD01'))

tmax_fig.add_trace(
    go.Bar(x=monthly.month,y=monthly.maxt,text=monthly.tmax_diff,name='Rocky Ford 2 ESE'))

tmax_fig.show()

AttributeError: 'DataFrame' object has no attribute 'tmax_diff'

In [19]:
df

,station,date,tmax,mxtm,tmin,mntm,tmax_time,tmin_time,month
0,rfd01,1992-06-04,76.2620,18:41:00,52.8980,01:24:00,1992-06-04 18:41:00,1992-06-04 01:24:00,6.0
1,rfd01,1992-06-05,85.3520,14:24:00,53.8880,05:40:00,1992-06-05 14:24:00,1992-06-05 05:40:00,6.0
2,rfd01,1992-06-06,76.2800,15:27:00,47.0840,05:34:00,1992-06-06 15:27:00,1992-06-06 05:34:00,6.0
3,rfd01,1992-06-07,84.9560,17:21:00,52.7900,06:22:00,1992-06-07 17:21:00,1992-06-07 06:22:00,6.0
4,rfd01,1992-06-08,76.4240,15:03:00,54.2300,02:57:00,1992-06-08 15:03:00,1992-06-08 02:57:00,6.0
5,rfd01,1992-06-09,68.5220,17:36:00,51.4040,00:00:00,1992-06-09 17:36:00,1992-06-09 00:00:00,6.0
6,rfd01,1992-06-10,80.6360,16:46:00,46.2200,05:21:00,1992-06-10 16:46:00,1992-06-10 05:21:00,6.0
7,rfd01,1992-06-11,77.9180,15:00:00,53.0420,18:18:00,1992-06-11 15:00:00,1992-06-11 18:18:00,6.0
8,rfd01,1992-06-12,80.7440,16:53:00,51.4220,05:43:00,1992-06-12 16:53:00,1992-06-12 05:43:00,6.0
9,rfd01,1992-06-13,88.6820,17:02:00,56.9480,04:11:00,1992-06-13 17:02:00,1992-06-13 04:11:00,6.0
